<a href="https://colab.research.google.com/github/BMaksim/boilerplate-removal/blob/main/boilerplate_rm_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dir = "/content/drive/My Drive/boilerplate_rm"

In [4]:
from bs4 import BeautifulSoup as bs
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopWords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def cl_string(text):
    text = ''.join([ch for ch in text if ch not in string.punctuation])
    text = text.lower()
    text  = ' '.join([w for w in text.split() if w not in stopWords])

    return text


X = []
y = []

for i in range(1, 800):
  try:
    with open(dir + f"/data/orig/{i}.html", "r", encoding='cp850') as file:
        content = file.read()
        soup = bs(content, "html.parser")
    text = re.sub('\n', '', soup.get_text('|'))
    text = re.sub(' +', ' ', text).split('|')
    text = list(map(cl_string, text))
    text = [el for el in text if el != '']
    text = [el for el in text if re.match(r'[0-9]+', el) is None]
    
    with open(dir + f'/data/clean/{i}.txt', 'r', encoding='cp850') as file:
      text1 = re.sub(' +', ' ', file.read()).split('\n\n')

    text1 = [re.sub('\n', '', el)[3:] for el in text1]
    text1 = list(map(cl_string, text1))
    text1 = [el for el in text1 if el != '']

    text0 = ' '.join(text1)
    res = [0 for i in range(len(text))]

    for i, el in enumerate(text):
        if el not in text0:
            res[i] = 1
            
    X.extend(text)
    y.extend(res)
  except FileNotFoundError:
    continue


In [8]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 11.3MB/s 
     |████████████████████████████████| 1.2MB 21.7MB/s 
     |████████████████████████████████| 901kB 41.4MB/s 
     |████████████████████████████████| 133kB 43.1MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 7.4MB 41.9MB/s 
ERROR: botocore 1.20.57 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=111)

In [10]:
import torch
import numpy as np
from pytorch_transformers import BertTokenizer, BertConfig
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from pytorch_transformers import AdamW, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case = True)
tokenized_data = [tokenizer.tokenize(sentence) for sentence in X_train]
input_ids = [tokenizer.convert_tokens_to_ids(token) for token in tokenized_data]
input_ids = pad_sequences(
    input_ids,
    maxlen=128,
    dtype="long",
    truncating="post",
    padding="post"
)

train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(y_train)

train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader( train_data, sampler = RandomSampler(train_data), batch_size = 32)

100%|██████████| 231508/231508 [00:00<00:00, 694062.48B/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (1680 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1138 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (982 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1785 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (759 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence lengt

In [12]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

100%|██████████| 440473133/440473133 [00:28<00:00, 15681075.40B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [13]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [14]:
%%time
model.train()

for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_labels = batch
    loss = model(b_input_ids, token_type_ids=None, labels=b_labels)

    optimizer.zero_grad()
    loss[0].backward()
    optimizer.step()

/usr/local/lib/python3.7/dist-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


CPU times: user 36min 19s, sys: 30min 18s, total: 1h 6min 38s
Wall time: 1h 6min 26s


In [15]:
tokenized_data = [tokenizer.tokenize(sentence) for sentence in X_test]
input_ids = [tokenizer.convert_tokens_to_ids(token) for token in tokenized_data]
input_ids = pad_sequences(
    input_ids,
    maxlen=128,
    dtype="long",
    truncating="post",
    padding="post"
)

prediction_inputs = torch.tensor(input_ids)
prediction_labels = torch.tensor(y_test)

prediction_data = TensorDataset(prediction_inputs, prediction_labels)

prediction_dataloader = DataLoader(prediction_data, sampler=SequentialSampler(prediction_data), batch_size=32)

Token indices sequence length is longer than the specified maximum sequence length for this model (5956 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (558 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (579 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (985 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (776 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

In [17]:
model.eval()
test_preds = []

for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_labels = batch

    with torch.no_grad():
        logits = model(b_input_ids, token_type_ids=None)

    logits = logits[0].detach().cpu().numpy()

    batch_preds = np.argmax(logits, axis=1)
    test_preds.extend(batch_preds)

In [19]:
def report(labels, preds):
    accuracy = accuracy_score(labels, preds)
    recall = recall_score(labels, preds, average="binary")
    f1 = f1_score(labels, preds, average="binary")
    precision = precision_score(labels, preds, average="binary")
    return {"accuracy": accuracy, "recall": recall, "f1": f1, "precision": precision}

result = report(y_test, test_preds)
print(result)

{'accuracy': 0.7593135203013813, 'recall': 0.6791400348634515, 'f1': 0.7175834970530452, 'precision': 0.7606403748535728}


In [20]:
len(X)

127411